# eda004
ペア裏削除による影響確認

In [13]:
MODE = 'local_train'
#MODE = 'kaggle_inference'

In [14]:
import os
import sys
import gc

if MODE == 'local_train':
    sys.path.append('/home/kaggler/.local/lib/python3.8/site-packages')
    from dotenv import load_dotenv
    load_dotenv
    sys.path.append(os.getenv('UTILS_PATH'))
    import line_notify
    import slack_notify
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.style.use("ggplot")
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import itertools
from tqdm import tqdm
from scipy.spatial.distance import canberra
from sklearn.neighbors import KNeighborsRegressor
import multiprocessing
import Levenshtein
import difflib
import pickle
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [15]:
# directry_setting
if MODE == 'local_train':
    INPUT_DIR = os.getenv('INPUT_DIR')
    OUTPUT_DIR = os.getenv('OUTPUT_DIR')
    MODEL_DIR = os.getenv('OUTPUT_DIR')
    #os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

elif MODE == 'kaggle_inference':
    INPUT_DIR = '/kaggle/input/foursquare-location-matching/'
    OUTPUT_DIR = './'
    MODEL_DIR = f'../input/fs{exp_name}/'

In [16]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame, org_data):
    scores = []
    id2poi = get_id2poi(org_data)
    poi2ids = get_poi2ids(org_data)
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def calc_max_score(tr_data, org_data):
    train_candidate = pd.DataFrame()
    train_candidate['id'] = org_data['id'].unique()
    train_candidate['matches'] = org_data['id'].unique()
    idx = tr_data['point_of_interest']==tr_data['match_point_of_interest']
    train_match = tr_data.loc[idx].groupby('id')['match_id'].apply(list).map(" ".join).reset_index()
    train_match.columns = ['id','candidates']
    train_candidate = train_candidate.merge(train_match, on = 'id', how = 'left')
    idx = ~train_candidate['candidates'].isna()
    train_candidate.loc[idx, "matches"] += " " + train_candidate.loc[idx, "candidates"]
    score = get_score(train_candidate, org_data)
    #print('1st_stage_max_score : ' + '{:.5f}'.format(score))
    return score

In [17]:
def add_orgin_data(df, org_df):
    df = df.merge(org_df.add_prefix('match_'), on='match_id')
    df = df.reset_index(drop=True)
    return df

In [18]:
# CONFIG
SEED = 42
N_NEIGHBORS = 10
N_SPLITS = 5
PROB_TH = 0.5

In [19]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame, org_data):
    scores = []
    id2poi = get_id2poi(org_data)
    poi2ids = get_poi2ids(org_data)
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def calc_max_score(tr_data, org_data):
    train_candidate = pd.DataFrame()
    train_candidate['id'] = org_data['id'].unique()
    train_candidate['matches'] = org_data['id'].unique()
    idx = tr_data['point_of_interest']==tr_data['match_point_of_interest']
    train_match = tr_data.loc[idx].groupby('id')['match_id'].apply(list).map(" ".join).reset_index()
    train_match.columns = ['id','candidates']
    train_candidate = train_candidate.merge(train_match, on = 'id', how = 'left')
    idx = ~train_candidate['candidates'].isna()
    train_candidate.loc[idx, "matches"] += " " + train_candidate.loc[idx, "candidates"]
    score = get_score(train_candidate, org_data)
    #print('1st_stage_max_score : ' + '{:.5f}'.format(score))
    return score

In [20]:
def transform_data(df, org_data):
    train_candidate = pd.DataFrame()
    train_candidate['id'] = org_data['id'].unique()
    train_candidate['matches'] = org_data['id'].unique()

    train_match = df[df['prob'] >= PROB_TH].copy()
    train_match = train_match.groupby('id')['match_id'].apply(list).map(" ".join).reset_index()
    train_match.columns = ['id','candidates']
    train_candidate = train_candidate.merge(train_match, on = 'id', how = 'left')
    idx = ~train_candidate['candidates'].isna()
    train_candidate.loc[idx, "matches"] += " " + train_candidate.loc[idx, "candidates"]
    return train_candidate[['id', 'matches']]


In [21]:
def postprocess(df):
    id2match = dict(zip(df["id"].values, df["matches"].str.split()))

    for match in tqdm(df["matches"]):
        match = match.split()
        if len(match) == 1:        
            continue

        base = match[0]
        for m in match[1:]:
            if not base in id2match[m]:
                id2match[m].append(base)
    df["matches"] = df["id"].map(id2match).map(" ".join)
    return df 

In [22]:
oof = pd.read_csv(OUTPUT_DIR + "exp006/exp006_oof.csv")
oof_dp = oof[~pd.DataFrame(np.sort(oof[['id','match_id']].values,1)).duplicated()].reset_index(drop=True)
train_origin = pd.read_csv(INPUT_DIR + "train.csv")

In [23]:
oof = transform_data(oof, train_origin)
cv_score = get_score(oof, train_origin)
print(f'cv_score = ' + '{:.5f}'.format(cv_score))
oof = postprocess(oof)
cv_score_after_pp = get_score(oof, train_origin)
print(f'cv_score(after_pp) = ' + '{:.5f}'.format(cv_score_after_pp))

cv_score = 0.83176


100%|██████████| 1138812/1138812 [00:01<00:00, 1077250.83it/s]


cv_score(after_pp) = 0.83792


In [24]:
oof_dp = transform_data(oof_dp, train_origin)
cv_score = get_score(oof_dp, train_origin)
print(f'cv_score = ' + '{:.5f}'.format(cv_score))
oof_dp = postprocess(oof_dp)
cv_score_after_pp = get_score(oof_dp, train_origin)
print(f'cv_score(after_pp) = ' + '{:.5f}'.format(cv_score_after_pp))

cv_score = 0.74285


100%|██████████| 1138812/1138812 [00:00<00:00, 1460989.41it/s]


cv_score(after_pp) = 0.83597


In [25]:
oof

,id,matches
0,E_000001272c6c5d,E_000001272c6c5d E_da7fa3963561f8
1,E_000002eae2a589,E_000002eae2a589 E_e80db432029aea
2,E_000007f24ebc95,E_000007f24ebc95
3,E_000008a8ba4f48,E_000008a8ba4f48
4,E_00001d92066153,E_00001d92066153 E_7e0d8e9138dd56
...,...,...
1138807,E_ffffb80854f713,E_ffffb80854f713 E_008558a7c44ed5
1138808,E_ffffbf9a83e0ba,E_ffffbf9a83e0ba E_37cbd58e31092a
1138809,E_ffffc572b4d35b,E_ffffc572b4d35b E_27bcc6f6dd33ed E_8081185a21...
1138810,E_ffffca745329ed,E_ffffca745329ed E_04988888cfff60


In [27]:
oof_dp

,id,matches
0,E_000001272c6c5d,E_000001272c6c5d E_da7fa3963561f8
1,E_000002eae2a589,E_000002eae2a589 E_e80db432029aea
2,E_000007f24ebc95,E_000007f24ebc95
3,E_000008a8ba4f48,E_000008a8ba4f48
4,E_00001d92066153,E_00001d92066153 E_7e0d8e9138dd56
...,...,...
1138807,E_ffffb80854f713,E_ffffb80854f713 E_008558a7c44ed5
1138808,E_ffffbf9a83e0ba,E_ffffbf9a83e0ba E_37cbd58e31092a
1138809,E_ffffc572b4d35b,E_ffffc572b4d35b E_27bcc6f6dd33ed E_8081185a21...
1138810,E_ffffca745329ed,E_ffffca745329ed E_04988888cfff60


In [29]:
oof["matches_dp"] = oof_dp["matches"]

In [31]:
oof[oof["matches"]!=oof["matches_dp"]]

,id,matches,matches_dp
10,E_00009ab517afac,E_00009ab517afac E_d290d38ca592e4 E_5731151815...,E_00009ab517afac E_d290d38ca592e4
18,E_000156e4ae899c,E_000156e4ae899c E_d28bf8b3338a91,E_000156e4ae899c
25,E_0001a397f67ad5,E_0001a397f67ad5 E_4b83e498f06eff,E_0001a397f67ad5
56,E_0003b15b0cc3b3,E_0003b15b0cc3b3 E_f154e5b6b26286,E_0003b15b0cc3b3
91,E_0005d223c299ab,E_0005d223c299ab E_811d75955cb441 E_3db8de706c...,E_0005d223c299ab E_811d75955cb441 E_3db8de706c...
...,...,...,...
1138765,E_fffd925d18b1c9,E_fffd925d18b1c9 E_b9f861a9362614 E_42737ba04c...,E_fffd925d18b1c9 E_b9f861a9362614 E_908a6ab867...
1138767,E_fffda1670277d8,E_fffda1670277d8 E_724d3a1107984a,E_fffda1670277d8
1138783,E_fffe5a71e05990,E_fffe5a71e05990 E_6aa8da4bb91ff5 E_132a2f7db2...,E_fffe5a71e05990 E_6aa8da4bb91ff5 E_132a2f7db2...
1138790,E_fffe9cbe5a2c75,E_fffe9cbe5a2c75 E_959fea1a5ee646 E_c9ff41f8d5...,E_fffe9cbe5a2c75 E_959fea1a5ee646 E_00fafe1b20...


In [32]:
oof_org = pd.read_csv(OUTPUT_DIR + "exp006/exp006_oof.csv")

In [37]:
_id = "E_ffff43afdebe13"
_id2 = "E_5a7cd9caae6451"
pd.concat([oof_org[(oof_org["id"]==_id)&(oof_org["match_id"]==_id2)],
           oof_org[(oof_org["id"]==_id2)&(oof_org["match_id"]==_id)]])

,id,match_id,target,prob
1918875,E_ffff43afdebe13,E_5a7cd9caae6451,1,0.474050
1918879,E_5a7cd9caae6451,E_ffff43afdebe13,1,0.564548
